In [19]:
!pip install datasets==2.15.0

In [1]:
import transformers

transformers.__version__

'4.35.0'

In [2]:
from datasets import Dataset
import datasets

datasets.__version__

'2.15.0'

创建数据集，如果出现错误，请升级datasets到2.15.0

In [16]:
from datasets import load_dataset


datasets = load_dataset(
    "text", 
    data_files={
        "train": 'C:\\Users\\owen\\Downloads\\中国通史\\中国通史.txt', 
        "validation": 'C:\\Users\\owen\\Downloads\\中国通史\\eval.txt'
    }
)
datasets["train"][10]
# def gen():
#     for x in open('C:\\Users\\owen\\Downloads\\中国通史\\中国通史.txt'):
#         x = x.strip('\n')
#         if not x:
#             continue
#         yield {'text': x}

# dataset = Dataset.from_generator(gen)
# dataset[1]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'text': '《诗·大雅》、《周颂》、《鲁颂》都有诗史性质的篇章。《生民》，歌咏周人始祖姜嫄及其子后稷的故事。《緜》歌咏古公亶父在周原创业的故事。《大明》歌咏殷周关系的变化。《大明》说，王季、文王都娶了殷商贵族的女儿。文王迎亲时，在渭水上“造舟为梁，不（丕）显其光”，举行了盛大的仪式。后来武王奉天命伐殷，大战牧野，军容很盛。这是一篇写得相当有系统的诗。《鲁颂·閟宫》，旧说是颂鲁僖公的功德。诗中，从姜嫄之德和后稷播百谷，说到太王、文王、武王相继，振兴国家，牧野克殷，再说到周公受封，建立鲁邦。诗中还说到鲁侯的武功，“戎狄是膺，荆舒是惩”，“淮夷蛮貊，及彼南夷，莫不率从。”《商颂》五篇，颂商的史事，也是从其始祖有娀氏因“天命玄鸟，降而生商”说起。诗中说到，“维女（汝）荆楚，居国南乡。昔有成汤，自彼氐羌，莫敢不来享，莫敢不来王。”殷周始祖各有诞生的奇迹，表明殷周各有自己的族源。荆蛮、淮夷、氐羌，还有玁狁，都是殷周以外的重要民族，各有自己的文化系统。'}

In [17]:
from transformers import AutoTokenizer

model_checkpoint = 'model_hub/gpt2-xl'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

print(tokenizer)


GPT2TokenizerFast(name_or_path='model_hub/gpt2-xl', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [18]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])
# tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map:   0%|          | 0/62560 [00:00<?, ? examples/s]

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

In [19]:
tokenized_datasets["train"][1]

{'input_ids': [10310,
  98,
  29826,
  96,
  18004,
  108,
  23626,
  236,
  171,
  120,
  248,
  17312,
  105,
  20046,
  99,
  10310,
  118,
  10310,
  104,
  10310,
  104,
  22887,
  237,
  46237,
  112,
  163,
  121,
  239,
  7,
  2503,
  13,
  1477,
  4669,
  88,
  86,
  13,
  535,
  8,
  21410,
  18796,
  101,
  22755,
  115,
  41468,
  27670,
  254,
  164,
  229,
  111,
  17739,
  114,
  28839,
  101,
  17312,
  105,
  44165,
  247,
  21410,
  27764,
  246,
  43636,
  101,
  163,
  102,
  118,
  29785,
  112,
  171,
  120,
  234,
  17312,
  105,
  44165,
  247,
  20998,
  103,
  162,
  237,
  238,
  160,
  122,
  249,
  51,
  25010,
  17739,
  101,
  37239,
  228,
  18796,
  113,
  36310,
  20046,
  99,
  27764,
  246,
  43636,
  101,
  17312,
  235,
  27950,
  94,
  20015,
  98,
  20998,
  232,
  17739,
  235,
  164,
  112,
  117,
  10310,
  233,
  164,
  121,
  121,
  17312,
  235,
  27950,
  94,
  171,
  120,
  234,
  20015,
  98,
  10310,
  233,
  43291,
  161,
  241,
  223,

In [20]:
block_size = 128

In [21]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [22]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    # num_proc=4,
)

Map:   0%|          | 0/62560 [00:00<?, ? examples/s]

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

In [23]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'下载服务，以下作品内容之版权与本站无任何关系。在线阅读：http://www.shuyyw.cc/read/4716/--------------------------------------------------1题记《中国通史》的编撰工作，是从一九七五年开始的。当时'

In [24]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [25]:
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('/miniconda/lib'), WindowsPath('E')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpyt

e:\miniconda\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
e:\miniconda\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: E:\miniconda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues